In [1]:
!pip install -q boto3 awswrangler pandas pyarrow sqlalchemy

In [2]:
import boto3
import awswrangler as wr
import pandas as pd
import os

In [17]:
AWS_REGION = "us-east-1"
S3_BUCKET = "kavita-hw21-spotify-datalake"
S3_RAW_PATH = f"s3://{S3_BUCKET}/raw/"
S3_PARQUET_PATH = f"s3://{S3_BUCKET}/parquet/"
DATABASE_NAME = "homework_2_1_db"
CSV_TABLE = "spotify_csv"
PARQUET_TABLE = "spotify_parquet"

session = boto3.Session(region_name=AWS_REGION)
s3 = session.client("s3")

try:
    if AWS_REGION == "us-east-1":
        s3.create_bucket(Bucket=S3_BUCKET)
    else:
        s3.create_bucket(
            Bucket=S3_BUCKET,
            CreateBucketConfiguration={
                "LocationConstraint": AWS_REGION
            }
        )
    print("Bucket created")
except Exception as e:
    print("Bucket already exists or error:", e)

Bucket created


In [18]:
LOCAL_DATASET_PATH = "aai-540-homework/homework-2-1/data/dataset.csv"

wr.s3.upload(
    local_file=LOCAL_DATASET_PATH,
    path=f"{S3_RAW_PATH}dataset.csv"
)

print("Dataset uploaded to S3")

Dataset uploaded to S3


In [19]:
wr.catalog.create_database(
    name=DATABASE_NAME,
    exist_ok=True
)

print("Athena database ready")

Athena database ready


In [20]:
df = pd.read_csv(LOCAL_DATASET_PATH)

wr.s3.to_csv(
    df=df,
    path=f"{S3_RAW_PATH}",
    dataset=True,
    database=DATABASE_NAME,
    table=CSV_TABLE,
    mode="overwrite"
)

print("CSV table registered in Athena")

2026-01-19 11:39:53,606	WARNING services.py:2070 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 409964544 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=0.69gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2026-01-19 11:39:53,779	INFO worker.py:1852 -- Started a local Ray instance.


CSV table registered in Athena


In [21]:
wr.s3.to_parquet(
    df=df,
    path=S3_PARQUET_PATH,
    dataset=True,
    database=DATABASE_NAME,
    table=PARQUET_TABLE,
    mode="overwrite"
)

print("Parquet table created")

Parquet table created


In [23]:
ATHENA_OUTPUT = f"s3://{S3_BUCKET}/athena-results/"

In [24]:
statement = f"""
SELECT track_name, energy
FROM {DATABASE_NAME}.{PARQUET_TABLE}
WHERE energy >= 0.5
"""

df_sql_energy = wr.athena.read_sql_query(
    sql=statement,
    database=DATABASE_NAME,
    s3_output=ATHENA_OUTPUT
)

df_sql_energy.head()

,track_name,energy
0,Hunger,0.632
1,Hold On - Remix,0.780
2,Falling in Love at a Coffee Shop,0.561
3,Unlonely,0.667
4,The Haves,0.519


In [25]:
df_pd_energy = df[df["energy"] >= 0.5][["track_name", "energy"]]
df_pd_energy.head()

,track_name,energy
9,Hunger,0.632
14,Hold On - Remix,0.780
15,Falling in Love at a Coffee Shop,0.561
24,Unlonely,0.667
40,The Haves,0.519


1. List artist, track_name, and popularity for songs that have a popularity greater than or equal to 99

In [39]:
query_1 = f"""
SELECT artists, track_name, popularity
FROM {DATABASE_NAME}.{PARQUET_TABLE}
WHERE popularity >= 99
"""

df_q1 = wr.athena.read_sql_query(
    sql=query_1,
    database=DATABASE_NAME,
    s3_output=ATHENA_OUTPUT
)

df_q1

,artists,track_name,popularity
0,Sam Smith;Kim Petras,Unholy (feat. Kim Petras),100
1,Bizarrap;Quevedo,"Quevedo: Bzrp Music Sessions, Vol. 52",99
2,Sam Smith;Kim Petras,Unholy (feat. Kim Petras),100


2. List artists with an average popularity of 92

In [31]:
query_2 = f"""
SELECT artists, AVG(popularity) AS avg_popularity
FROM {DATABASE_NAME}.{PARQUET_TABLE}
GROUP BY artists
HAVING AVG(popularity) = 92
"""

df_q2 = wr.athena.read_sql_query(
    sql=query_2,
    database=DATABASE_NAME,
    s3_output=ATHENA_OUTPUT
)

df_q2

,artists,avg_popularity
0,Harry Styles,92.0
1,Rema;Selena Gomez,92.0


3. List the Top 10 genres with the highest average energy

In [34]:
query_3 = f"""
SELECT track_genre, AVG(energy) AS avg_energy
FROM {DATABASE_NAME}.{PARQUET_TABLE}
GROUP BY track_genre
ORDER BY avg_energy DESC
LIMIT 10
"""

df_q3 = wr.athena.read_sql_query(
    sql=query_3,
    database=DATABASE_NAME,
    s3_output=ATHENA_OUTPUT
)

df_q3

,track_genre,avg_energy
0,death-metal,0.931470
1,grindcore,0.924201
2,metalcore,0.914485
3,happy,0.910971
4,hardstyle,0.901246
5,drum-and-bass,0.876635
6,black-metal,0.874897
7,heavy-metal,0.874003
8,party,0.871237
9,j-idol,0.868677


4. How many tracks is Bad Bunny on?

In [36]:
query_4 = f"""
SELECT COUNT(*) AS track_count
FROM {DATABASE_NAME}.{PARQUET_TABLE}
WHERE artists = 'Bad Bunny'
"""

df_q4 = wr.athena.read_sql_query(
    sql=query_4,
    database=DATABASE_NAME,
    s3_output=ATHENA_OUTPUT
)

df_q4

,track_count
0,48


5. Show the top 10 genres in terms of popularity, sorted by their most popular track

In [38]:
query_5 = f"""
SELECT track_genre, MAX(popularity) AS max_popularity
FROM {DATABASE_NAME}.{PARQUET_TABLE}
GROUP BY track_genre
ORDER BY max_popularity DESC
LIMIT 10
"""

df_q5 = wr.athena.read_sql_query(
    sql=query_5,
    database=DATABASE_NAME,
    s3_output=ATHENA_OUTPUT
)

df_q5

,track_genre,max_popularity
0,pop,100
1,dance,100
2,hip-hop,99
3,latin,98
4,edm,98
5,reggaeton,98
6,reggae,98
7,latino,98
8,rock,96
9,piano,96
